In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers -q
!pip install llama_index -q
%pip install llama-index-llms-huggingface
!pip install --upgrade transformers

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
%%time

documents = SimpleDirectoryReader("./data/scraped").load_data()

In [ ]:
len(documents), documents[0]

In [ ]:
system_prompt="""
You are a personal Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:

!huggingface-cli login 

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import torch

# Manually assign the token (replace with your actual token)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.8, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="stabilityai/stablelm-zephyr-3b",
    model_name="stabilityai/stablelm-zephyr-3b",
    # device_map="cuda" if torch.cuda.is_available() else "cpu",  # Explicitly set device
    device_map="balanced",  # Explicitly set device
    model_kwargs={"torch_dtype": torch.float16, "load_in_4bit": True, 'token': token}
)


In [ ]:
%pip install llama-index-embeddings-langchain


In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:
import gc
gc.collect()

In [14]:
from llama_index.core.settings import Settings

Settings.chunk_size = 1024
Settings.llm = llm
Settings.embed_model = embed_model

In [15]:
index = VectorStoreIndex.from_documents(documents)

In [16]:
query_engine=index.as_query_engine()

In [ ]:
response = query_engine.query("What is the basic salary of a data scientist")
print(response)

In [ ]:
!pip install Gradio

In [19]:
# import gradio as gr
# from collections import deque

# # Multi-Turn Memory
# conversation_history = deque(maxlen=5)

# def chatbot(input_text):
#     """Handles chatbot queries with multi-turn memory."""
#     conversation_history.append(f"👤 User: {input_text}")
#     full_context = "\n".join(conversation_history)

#     # Replace this with your actual chatbot response mechanism
#     response = query_engine.query(full_context)  # Ensure query_engine is initialized

#     conversation_history.append(f"🤖 Assistant: {response}")
    
#     # Return formatted conversation
#     return "\n\n".join(conversation_history)

# def clear_chat():
#     """Clears the conversation history."""
#     conversation_history.clear()
#     return ""

# # Custom CSS for Styling
# custom_css = """
# /* Page background */
# /* Page background */
# body {
#     background-color: #2c2d2f;
#     font-family: Arial, sans-serif !important;
# }

# /* Chatbox styling */
# textarea {
#     font-size: 16px !important;
#     border-radius: 8px !important;
#     background: #ffffff !important;
#     border: 1px solid #ccc !important;
#     padding: 12px !important;
#     color: black !important;
#     resize: none !important; /* Prevents resizing for better UI */
# }

# /* User input box */
# input[type="text"], textarea {
#     color: white !important;
#     font-size: 16px !important;
#     border-radius: 8px !important;
#     border: 1px solid #666 !important;
#     padding: 12px !important;
#     background: #222222 !important; /* Dark background for contrast */
#     transition: all 0.3s ease-in-out !important;
# }

# /* User input box on focus */
# input[type="text"]:focus, textarea:focus {
#     border-color: #007bff !important;
#     outline: none !important;
#     background: #333333 !important;
# }

# /* Buttons */
# button {
#     font-size: 16px !important;
#     border-radius: 8px !important;
#     padding: 12px 18px !important;
#     border: none !important;
#     cursor: pointer !important;
#     font-weight: bold !important;
#     text-transform: uppercase !important;
#     transition: all 0.3s ease-in-out !important;
# }

# /* Send button */
# button:nth-child(1) {
#     background-color: #007bff !important;
#     color: white !important;
#     box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.2) !important;
# }

# button:nth-child(1):hover {
#     background-color: #0056b3 !important;
#     transform: scale(1.05) !important;
# }

# /* Clear button */
# button:nth-child(2) {
#     background-color: #dc3545 !important;
#     color: white !important;
#     box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.2) !important;
# }

# button:nth-child(2):hover {
#     background-color: #a71d2a !important;
#     transform: scale(1.05) !important;
# }

# /* Improved layout and wider spacing */
# .gradio-container {
#     max-width: 1600px !important; /* Increased width */
#     width: 90% !important; /* Make it responsive */
#     margin: auto !important;
#     padding: 30px !important;
#     border-radius: 12px !important;
#     background: rgb(35, 34, 34) !important;
#     box-shadow: 0px 6px 12px rgba(0, 0, 0, 0.15) !important;
# }



# """

# # Gradio UI with Custom CSS
# with gr.Blocks(css=custom_css) as demo:
#     gr.Markdown("# 🤖 **Smart AI Chatbot**", elem_id="title")
#     gr.Markdown("### 💬 Chat with an intelligent assistant!", elem_id="subtitle")

#     chatbox = gr.Textbox(label="Conversation History", interactive=False, lines=10)
#     user_input = gr.Textbox(lines=2, placeholder="Type your message...", label="Your Message")
    
#     with gr.Row():
#         submit_btn = gr.Button("🚀 Send")
#         clear_btn = gr.Button("🗑️ Clear Chat")

#     submit_btn.click(chatbot, inputs=user_input, outputs=chatbox)
#     clear_btn.click(clear_chat, outputs=chatbox)

# demo.launch()


In [ ]:
import gradio as gr
import requests
from bs4 import BeautifulSoup
import time
from collections import deque

# Multi-Turn Memory
conversation_history = deque(maxlen=5)

# Configure headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# ===== CHATBOT FUNCTIONS =====
def chatbot(input_text):
    """Handles chatbot queries with multi-turn memory."""
    conversation_history.append(f"👤 User: {input_text}")
    full_context = "\n".join(conversation_history)

    # Replace this with your actual chatbot response mechanism
    try:
        response = query_engine.query(full_context)  # Ensure query_engine is initialized
    except:
        # Fallback if query_engine is not available
        response = f"You said: {input_text}"

    conversation_history.append(f"🤖 Assistant: {response}\n")
    
    # Return formatted conversation with single newlines instead of double
    return "\n".join(conversation_history)

def clear_chat():
    """Clears the conversation history."""
    conversation_history.clear()
    return ""

# ===== LINKEDIN JOB SEARCH FUNCTIONS =====
def scrape_linkedin_jobs(query, location=""):
    """Scrape LinkedIn job listings based on search query"""
    base_url = "https://www.linkedin.com/jobs/search/"
    params = {
        "keywords": query,
        "location": location,
        "position": 1,
        "pageNum": 0
    }
    
    try:
        response = requests.get(base_url, params=params, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        jobs = []
        job_listings = soup.find_all('div', class_='base-card')
        
        for job in job_listings[:10]:  # Limit to 10 results
            title = job.find('h3', class_='base-search-card__title').text.strip()
            company = job.find('a', class_='hidden-nested-link').text.strip()
            job_location = job.find('span', class_='job-search-card__location').text.strip()
            link = job.find('a', class_='base-card__full-link')['href']
            
            jobs.append({
                'title': title,
                'company': company,
                'location': job_location,
                'link': link.split('?')[0]  # Clean URL
            })
            
            time.sleep(0.5)  # Be polite with requests
            
        return jobs
    
    except Exception as e:
        return f"Error: {str(e)}"

def display_jobs(query, location):
    # Validate query to ensure it's job-related
    if not is_valid_job_query(query):
        return "<div style='color: #FF6B6B; background-color: rgba(255, 107, 107, 0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #FF6B6B;'><strong>⚠️ Invalid Job Query:</strong> Please enter a valid job title, role, or industry. Examples: 'Software Engineer', 'Data Scientist', 'Marketing', 'Healthcare'.</div>"
    
    jobs = scrape_linkedin_jobs(query, location)
    
    if isinstance(jobs, str):  # Error case
        return f"<div style='color: red'>{jobs}</div>"
    
    if not jobs:
        return "<div style='color: orange'>No jobs found for this search.</div>"
    
    html_output = f"<h3>Found {len(jobs)} jobs:</h3>"
    for idx, job in enumerate(jobs, 1):
        html_output += f"""
        <div style='margin-bottom: 20px; border-bottom: 1px solid #ccc; padding-bottom: 10px;'>
            <h4>{idx}. {job['title']}</h4>
            <p><strong>Company:</strong> {job['company']}</p>
            <p><strong>Location:</strong> {job['location']}</p>
            <p><a href="{job['link']}" target="_blank">View Job</a></p>
        </div>
        """
    return html_output

def is_valid_job_query(query):
    """
    Validate if the query appears to be job-related.
    Returns True for valid job queries, False otherwise.
    """
    if not query or len(query.strip()) < 2:
        return False
        
    # Common job-related terms/prefixes/suffixes
    job_related_terms = [
        "developer", "engineer", "manager", "assistant", "specialist", "analyst", 
        "designer", "consultant", "coordinator", "director", "technician", "representative",
        "administrator", "supervisor", "officer", "programmer", "scientist", "associate",
        "intern", "job", "career", "position", "role", "hiring", "recruitment",
        "full-time", "part-time", "remote", "work", "employment",
        # Industries
        "tech", "it", "software", "health", "medical", "finance", "banking", "education",
        "retail", "sales", "marketing", "hr", "legal", "media", "design", "construction",
        "manufacturing", "engineering", "science", "research", "data", "ai", "ml",
        # Roles
        "ceo", "cto", "cfo", "vp", "head", "lead", "junior", "senior", "mid", "staff"
    ]
    
    query_lower = query.lower()
    
    # Check if any job-related term is in the query
    for term in job_related_terms:
        if term in query_lower or query_lower in term:
            return True
            
    # If query is very long, likely not a job search
    if len(query_lower) > 50:
        return False
        
    # If query contains question marks or specific non-job phrases
    if "?" in query or "who is" in query_lower or "what is" in query_lower or "how to" in query_lower:
        return False
        
    # Default to allowing the query if we're not sure
    return True

# Custom CSS for Styling
custom_css = """
/* Modern Gradient-based Color Scheme */
:root {
    --primary: #7C4DFF;    /* Deep purple */
    --secondary: #2A2E45;  /* Navy blue */
    --background: #1A1C28; /* Space black */
    --accent: #FF6B6B;     /* Coral pink */
    --text: #F0F0FF;       /* Soft lavender */
}

/* Base Styling with Larger Fonts */
body {
    font-size: 19px !important;
    line-height: 1.7 !important;
    background: linear-gradient(135deg, var(--background) 0%, #242736 100%);
    font-family: 'Inter', system-ui, sans-serif;
    color: var(--text) !important;
}

/* Text Elements Scaling */
h1 { font-size: 2.8rem !important; }
h2 { font-size: 2.2rem !important; }
h3 { font-size: 1.9rem !important; }
p { font-size: 1.1em !important; }

/* Chat Interface */
.gr-box {
    background: rgba(42, 46, 69, 0.85) !important;
    backdrop-filter: blur(12px);
    border-radius: 18px;
    border: 1px solid rgba(124, 77, 255, 0.25);
    font-size: 1.15rem !important;
}

/* Reduced gap between buttons */
.button-row {
    gap: 5px !important;
}

/* Conversation text spacing */
.gr-textbox span {
    line-height: 1.4 !important;
    margin-bottom: 0.3rem !important;
}

/* Chat display customization */
#chat-display {
    padding: 8px !important;
    margin-bottom: 10px !important;
}

#chat-display textarea {
    line-height: 1.3 !important;
}

/* Input Fields */
input[type="text"], textarea {
    font-size: 1.2rem !important;
    padding: 18px !important;
    background: rgba(255, 255, 255, 0.08) !important;
    border: 2px solid rgba(124, 77, 255, 0.3) !important;
}

/* Buttons */
button {
    font-size: 1.25rem !important;
    padding: 16px 28px !important;
    background-image: linear-gradient(135deg, var(--primary) 0%, #906BFF 100%) !important;
    border-radius: 14px !important;
}

button:hover {
    transform: translateY(-2px) scale(1.02);
    box-shadow: 0 10px 20px rgba(124, 77, 255, 0.25) !important;
}

/* Job Listings */
.job-card {
    background: linear-gradient(145deg, #2A2E45 0%, #1F2235 100%);
    border-radius: 16px;
    padding: 24px;
    font-size: 1.1rem !important;
}

.job-card h4 {
    font-size: 1.5rem !important;
    background: linear-gradient(45deg, var(--primary), var(--accent));
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}

/* Chat History */
#chat-history {
    font-size: 1.2rem !important;
    line-height: 1.8 !important;
    padding: 24px !important;
}

/* Interactive Elements */
.gr-textbox:focus, button:focus {
    box-shadow: 0 0 0 4px rgba(124, 77, 255, 0.3) !important;
}

/* Scrollbar */
::-webkit-scrollbar {
    width: 10px;
    background: var(--secondary);
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(var(--primary), var(--accent));
    border-radius: 6px;
}

/* Section Dividers */
.section-divider {
    height: 4px;
    background: linear-gradient(90deg, transparent 0%, var(--primary) 50%, transparent 100%);
    margin: 2.5rem 0;
}

/* Status Messages */
.gr-label {
    font-size: 1.3rem !important;
    letter-spacing: 0.5px;
}

/* Responsive Scaling */
@media (max-width: 768px) {
    body { font-size: 17px !important; }
    h1 { font-size: 2.2rem !important; }
    button { font-size: 1.1rem !important; }
}
"""

# Create the combined Gradio interface
with gr.Blocks(css=custom_css, title="AI Assistant with Job Search") as demo:
    gr.Markdown("# 🤖 **Smart AI Chatbot **", elem_id="title")
    
    # Chatbot Section (Top)
    with gr.Group():
        gr.Markdown("### 💬 Chat with an intelligent assistant!", elem_id="subtitle")
        
        chatbox = gr.Textbox(label="Conversation History", interactive=False, lines=12, elem_id="chat-display")
        user_input = gr.Textbox(lines=1, placeholder="Type your message...", label="Your Message")
        
        with gr.Row(elem_classes=["button-row"]):
            submit_btn = gr.Button("🚀 Send")
            clear_btn = gr.Button("🗑️ Clear Chat")

        submit_btn.click(fn=chatbot, inputs=user_input, outputs=chatbox)
        user_input.submit(fn=chatbot, inputs=user_input, outputs=chatbox)
        clear_btn.click(fn=clear_chat, outputs=chatbox)
    
    # Divider
    gr.Markdown("---", elem_id="divider", elem_classes=["section-divider"])
    
    # LinkedIn Job Search Section (Bottom)
    with gr.Group():
        gr.Markdown("## 🔍 LinkedIn Job Search", elem_id="subtitle")
        gr.Markdown("Enter your job search query below:")
        
        with gr.Row():
            job_query = gr.Textbox(label="Job title or keywords", placeholder="Software Engineer")
            job_location = gr.Textbox(label="Location (optional)", placeholder="New York")
        
        search_btn = gr.Button("🔍 Search Jobs")
        
        job_output = gr.HTML()
        
        search_btn.click(
            fn=display_jobs,
            inputs=[job_query, job_location],
            outputs=job_output,
        )

if __name__ == "__main__":
    demo.launch() 